<a href="https://www.kaggle.com/code/chhelp/spaceship-titanic?scriptVersionId=149271328" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/spaceship-titanic/sample_submission.csv
/kaggle/input/spaceship-titanic/train.csv
/kaggle/input/spaceship-titanic/test.csv


# [portal](https://www.kaggle.com/code/yinn94/try1-rf)

In [2]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix, classification_report, f1_score, log_loss
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.impute import SimpleImputer
rf = RandomForestClassifier()
svc = SVC()
xgc = GradientBoostingClassifier()
ss = StandardScaler() # 去均值和方差归一化，针对每一个特征维度来做，而不是针对样本
Encoder = OneHotEncoder()

In [3]:
data = pd.read_csv(r"/kaggle/input/spaceship-titanic/train.csv") # 读数据
data

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
0,0001_01,Europa,False,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,False
1,0002_01,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,Juanna Vines,True
2,0003_01,Europa,False,A/0/S,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,Altark Susent,False
3,0003_02,Europa,False,A/0/S,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,Solam Susent,False
4,0004_01,Earth,False,F/1/S,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,Willy Santantines,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8688,9276_01,Europa,False,A/98/P,55 Cancri e,41.0,True,0.0,6819.0,0.0,1643.0,74.0,Gravior Noxnuther,False
8689,9278_01,Earth,True,G/1499/S,PSO J318.5-22,18.0,False,0.0,0.0,0.0,0.0,0.0,Kurta Mondalley,False
8690,9279_01,Earth,False,G/1500/S,TRAPPIST-1e,26.0,False,0.0,0.0,1872.0,1.0,0.0,Fayey Connon,True
8691,9280_01,Europa,False,E/608/S,55 Cancri e,32.0,False,0.0,1049.0,0.0,353.0,3235.0,Celeon Hontichre,False


In [4]:
new_data = data.copy() # To avoid we manipulte the original data
new_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8693 entries, 0 to 8692
Data columns (total 14 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   PassengerId   8693 non-null   object 
 1   HomePlanet    8492 non-null   object 
 2   CryoSleep     8476 non-null   object 
 3   Cabin         8494 non-null   object 
 4   Destination   8511 non-null   object 
 5   Age           8514 non-null   float64
 6   VIP           8490 non-null   object 
 7   RoomService   8512 non-null   float64
 8   FoodCourt     8510 non-null   float64
 9   ShoppingMall  8485 non-null   float64
 10  Spa           8510 non-null   float64
 11  VRDeck        8505 non-null   float64
 12  Name          8493 non-null   object 
 13  Transported   8693 non-null   bool   
dtypes: bool(1), float64(6), object(7)
memory usage: 891.5+ KB


In [5]:
new_data.keys()
new_data[:20] # 显示前 20 行数据

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
0,0001_01,Europa,False,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,False
1,0002_01,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,Juanna Vines,True
2,0003_01,Europa,False,A/0/S,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,Altark Susent,False
3,0003_02,Europa,False,A/0/S,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,Solam Susent,False
4,0004_01,Earth,False,F/1/S,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,Willy Santantines,True
5,0005_01,Earth,False,F/0/P,PSO J318.5-22,44.0,False,0.0,483.0,0.0,291.0,0.0,Sandie Hinetthews,True
6,0006_01,Earth,False,F/2/S,TRAPPIST-1e,26.0,False,42.0,1539.0,3.0,0.0,0.0,Billex Jacostaffey,True
7,0006_02,Earth,True,G/0/S,TRAPPIST-1e,28.0,False,0.0,0.0,0.0,0.0,NaN,Candra Jacostaffey,True
8,0007_01,Earth,False,F/3/S,TRAPPIST-1e,35.0,False,0.0,785.0,17.0,216.0,0.0,Andona Beston,True
9,0008_01,Europa,True,B/1/P,55 Cancri e,14.0,False,0.0,0.0,0.0,0.0,0.0,Erraiam Flatic,True


In [6]:
new_data = new_data.dropna() # 删除具有 Null 值的行/列
cabin_np = np.array([])


for c in new_data['Cabin']:
    if type(c) == float:
        cabin_np = np.append(cabin_np, c)
    else:
        cabin_np = np.append(cabin_np, c.split('/')) # 指定分割符进行切片

cabin_np = cabin_np.reshape(int(len(cabin_np) / 3), 3)

print(cabin_np)

# 对数据进行更新
new_data['Cabin_deck'] = cabin_np[:,0]
new_data['Cabin_num'] = cabin_np[:,1]
new_data['Cabin_side'] = cabin_np[:,2]
new_data['All_cost'] = new_data[['RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck']].sum(axis = 1)
last_name = []

for n in new_data['Name']:
    last_name.append(n.split())
last_name_np = np.array(last_name)
new_data['Last_name'] = last_name_np[:,1]

new_data

[['B' '0' 'P']
 ['F' '0' 'S']
 ['A' '0' 'S']
 ...
 ['G' '1500' 'S']
 ['E' '608' 'S']
 ['E' '608' 'S']]


,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported,Cabin_deck,Cabin_num,Cabin_side,All_cost,Last_name
0,0001_01,Europa,False,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,False,B,0,P,0.0,Ofracculy
1,0002_01,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,Juanna Vines,True,F,0,S,736.0,Vines
2,0003_01,Europa,False,A/0/S,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,Altark Susent,False,A,0,S,10383.0,Susent
3,0003_02,Europa,False,A/0/S,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,Solam Susent,False,A,0,S,5176.0,Susent
4,0004_01,Earth,False,F/1/S,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,Willy Santantines,True,F,1,S,1091.0,Santantines
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8688,9276_01,Europa,False,A/98/P,55 Cancri e,41.0,True,0.0,6819.0,0.0,1643.0,74.0,Gravior Noxnuther,False,A,98,P,8536.0,Noxnuther
8689,9278_01,Earth,True,G/1499/S,PSO J318.5-22,18.0,False,0.0,0.0,0.0,0.0,0.0,Kurta Mondalley,False,G,1499,S,0.0,Mondalley
8690,9279_01,Earth,False,G/1500/S,TRAPPIST-1e,26.0,False,0.0,0.0,1872.0,1.0,0.0,Fayey Connon,True,G,1500,S,1873.0,Connon
8691,9280_01,Europa,False,E/608/S,55 Cancri e,32.0,False,0.0,1049.0,0.0,353.0,3235.0,Celeon Hontichre,False,E,608,S,4637.0,Hontichre


In [7]:
new_data.keys()

Index(['PassengerId', 'HomePlanet', 'CryoSleep', 'Cabin', 'Destination', 'Age',
       'VIP', 'RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck',
       'Name', 'Transported', 'Cabin_deck', 'Cabin_num', 'Cabin_side',
       'All_cost', 'Last_name'],
      dtype='object')

In [8]:
# fit_transform() 先拟合数据，再标准化
scaled_data = ss.fit_transform(new_data[['Age', 'RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck', 'All_cost']])
one_hot_data = Encoder.fit_transform(new_data[['HomePlanet', 'CryoSleep', 'Destination', 'VIP', 'Cabin_deck', 'Cabin_side']])
one_hot_data_np = one_hot_data.toarray()
target = new_data['Transported'].values

# concatenate() 沿着指定轴拼接一系列数组
x = np.concatenate((scaled_data, one_hot_data_np), axis = 1)

# train_test_split() 是交叉验证中常用的函数，功能是从样本中随机的按比例选取 train_data 和 test_data
x_train, x_test, y_train, y_test = train_test_split(x, target, test_size = .2)
x_train.shape

(5284, 27)

In [9]:
def train_model_performance(model, train_x, test_x, train_y, test_y, class_name):
    train_model = model.fit(train_x, train_y)
    pred = train_model.predict(test_x)
    print(classification_report(test_y, pred, target_names = class_name)) 
    print('F1 scroe overall:', round(f1_score(test_y,pred, average = 'micro'), 4))
    print(f'log_loss:{round(log_loss(test_y, pred), 4)}')
    cm = confusion_matrix(test_y, pred)
    print('-' * 100)

In [10]:
train_model_performance(svc, x_train, x_test, y_train, y_test, ['True', 'False'])
train_model_performance(rf, x_train, x_test, y_train, y_test, ['True', 'False'])
train_model_performance(xgc, x_train, x_test, y_train, y_test, ['True', 'False'])

              precision    recall  f1-score   support

        True       0.84      0.80      0.82       676
       False       0.80      0.84      0.82       646

    accuracy                           0.82      1322
   macro avg       0.82      0.82      0.82      1322
weighted avg       0.82      0.82      0.82      1322

F1 scroe overall: 0.8185
log_loss:6.5435
----------------------------------------------------------------------------------------------------
              precision    recall  f1-score   support

        True       0.79      0.84      0.82       676
       False       0.82      0.77      0.80       646

    accuracy                           0.81      1322
   macro avg       0.81      0.81      0.81      1322
weighted avg       0.81      0.81      0.81      1322

F1 scroe overall: 0.8086
log_loss:6.8979
----------------------------------------------------------------------------------------------------
              precision    recall  f1-score   support

       

In [11]:
test_data = pd.read_csv(r'/kaggle/input/spaceship-titanic/test.csv')
new_test_data = test_data.copy() # To avoid we manipulte the original data

new_test_data.info(), new_test_data.keys()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4277 entries, 0 to 4276
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   PassengerId   4277 non-null   object 
 1   HomePlanet    4190 non-null   object 
 2   CryoSleep     4184 non-null   object 
 3   Cabin         4177 non-null   object 
 4   Destination   4185 non-null   object 
 5   Age           4186 non-null   float64
 6   VIP           4184 non-null   object 
 7   RoomService   4195 non-null   float64
 8   FoodCourt     4171 non-null   float64
 9   ShoppingMall  4179 non-null   float64
 10  Spa           4176 non-null   float64
 11  VRDeck        4197 non-null   float64
 12  Name          4183 non-null   object 
dtypes: float64(6), object(7)
memory usage: 434.5+ KB


(None,
 Index(['PassengerId', 'HomePlanet', 'CryoSleep', 'Cabin', 'Destination', 'Age',
        'VIP', 'RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck',
        'Name'],
       dtype='object'))

In [12]:
object_col = ['HomePlanet', 'CryoSleep', 'Cabin', 'Destination', 'VIP']
numeric_col = ['Age', 'RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck']
imputer_numeric = SimpleImputer(strategy='mean')
imputer_obj = SimpleImputer(strategy='most_frequent')
imputer_num_data = imputer_numeric.fit_transform(new_test_data[numeric_col]) 
imputer_obj_data = imputer_obj.fit_transform(new_test_data[object_col]) 

imputer_after_data = np.concatenate((imputer_num_data, imputer_obj_data), axis = 1)

imputer_pd = pd.DataFrame(imputer_after_data, columns = [numeric_col + object_col])
imputer_pd['PassengerId'] = new_test_data['PassengerId']
imputer_pd['All_cost'] = imputer_pd[numeric_col[1:]].sum(axis = 1)

In [13]:
Cabin_deck = []
Cabin_num = []
Cabin_side = []
for c in imputer_pd['Cabin'].values:
    Cabin_deck.append(c[0][0])
    Cabin_num.append(c[0][2])
    Cabin_side.append(c[0][-1])

imputer_pd['Cabin_deck'] = Cabin_deck
imputer_pd['Cabin_num'] = Cabin_num
imputer_pd['Cabin_side'] = Cabin_side
imputer_pd['Cabin_side'].value_counts()

(Cabin_side,)
P                2184
S                2093
Name: count, dtype: int64

In [14]:
scaled_data = ss.fit_transform(imputer_pd[['Age','RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck', 'All_cost']])
one_hot_data = Encoder.fit_transform(imputer_pd[['HomePlanet', 'CryoSleep', 'Destination', 'VIP', 'Cabin_deck', 'Cabin_side']])
one_hot_data_np = one_hot_data.toarray()
one_hot_data_np.shape
x = np.concatenate((scaled_data, one_hot_data_np), axis = 1)
x.shape

(4277, 27)

In [15]:
pred = rf.predict(x)

submission = pd.read_csv(r'/kaggle/input/spaceship-titanic/sample_submission.csv')

In [16]:
submission['Transported'] = pred
submission
submission.to_csv(r'/kaggle/working/prediction.csv', index = False)